# Import Library yang Digunakan

In [123]:
# import library yang digunakan

import pandas as pd 
import string
import re
import emoji
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from indoNLP.preprocessing import replace_slang, replace_word_elongation

# stopwords removal
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LARAS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\LARAS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Import Dataset

In [110]:
data_df =  pd.read_csv('./youtube_comments.csv')
data_df.head()

,author,published_time,likes,comment,isPublic
0,@ferryirwandi,2024-10-10T09:02:59Z,4729,"hallo warga sipil sekalian, selamat menonton",True
1,@udinaja5394,2025-04-11T22:47:18Z,0,prestasi menteri pendidikan LGBT ala barat,True
2,@anisafiqriyah8612,2025-04-10T06:09:05Z,0,Semoga Mentri Pendidikan periode ini bisa meng...,True
3,@MuhammadAmin-yt8sd,2025-04-09T15:59:13Z,0,"Jgn ngerokok bos, minimal di video ngga keliat...",True
4,@faridfarid-kv2il,2025-04-09T01:56:26Z,0,Gen produk nadiem😅😅😅,True


## Menilai Data

In [111]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12142 entries, 0 to 12141
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   author          12139 non-null  object
 1   published_time  12142 non-null  object
 2   likes           12142 non-null  int64 
 3   comment         12142 non-null  object
 4   isPublic        12142 non-null  bool  
dtypes: bool(1), int64(1), object(3)
memory usage: 391.4+ KB


In [112]:
data_df.describe(include='all')

,author,published_time,likes,comment,isPublic
count,12139,12142,12142.000000,12142,12142
unique,11470,11997,NaN,12096,1
top,@LLLLlll__0000,2024-10-11T03:06:20Z,NaN,😢,True
freq,131,3,NaN,9,12142
mean,NaN,NaN,5.991929,NaN,NaN
std,NaN,NaN,187.515345,NaN,NaN
min,NaN,NaN,0.000000,NaN,NaN
25%,NaN,NaN,0.000000,NaN,NaN
50%,NaN,NaN,0.000000,NaN,NaN
75%,NaN,NaN,0.000000,NaN,NaN


In [113]:
# melihat missing value
cleaned_df.isnull().sum()

comment    0
dtype: int64

In [114]:
# melihat data duplikat
cleaned_df[cleaned_df.duplicated(subset='comment')]

,comment
501,pasti yang nonton orang pintar
723,😢
905,bicara pendidikan ko sambil merokok bodoh kamu...
907,bicara pendidikan ko sambil merokok bodoh kamu...
981,"yang bego nya itu, dulu waktu sd 1 kelas yang ..."
...,...
11909,😢
11922,up
12081,indonesia (c)emas 2045
12135,❤


In [115]:
# menghapus comment duplikat
cleaned_df = cleaned_df.drop_duplicates(subset='comment')

In [116]:
# melihat jumlah data setelah duplikasi di hapus
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12077 entries, 0 to 12141
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  12077 non-null  object
dtypes: object(1)
memory usage: 188.7+ KB


- tidak ditemukan missing value pada hasil scraping comment
- terdapat beberapa data duplikat pada kolom 'comment' 

## Data Cleaning

In [117]:
# copy data
cleaned_df = data_df.copy()
cleaned_df.head()

,author,published_time,likes,comment,isPublic
0,@ferryirwandi,2024-10-10T09:02:59Z,4729,"hallo warga sipil sekalian, selamat menonton",True
1,@udinaja5394,2025-04-11T22:47:18Z,0,prestasi menteri pendidikan LGBT ala barat,True
2,@anisafiqriyah8612,2025-04-10T06:09:05Z,0,Semoga Mentri Pendidikan periode ini bisa meng...,True
3,@MuhammadAmin-yt8sd,2025-04-09T15:59:13Z,0,"Jgn ngerokok bos, minimal di video ngga keliat...",True
4,@faridfarid-kv2il,2025-04-09T01:56:26Z,0,Gen produk nadiem😅😅😅,True


In [118]:
# menghapus kolom yang tidak diperlukan
cleaned_df.drop(columns=['author','published_time','likes', 'isPublic'], inplace=True)
cleaned_df.head()

,comment
0,"hallo warga sipil sekalian, selamat menonton"
1,prestasi menteri pendidikan LGBT ala barat
2,Semoga Mentri Pendidikan periode ini bisa meng...
3,"Jgn ngerokok bos, minimal di video ngga keliat..."
4,Gen produk nadiem😅😅😅


### Text Pre-processing

In [119]:
# case folding
cleaned_df['comment'] = cleaned_df['comment'].str.lower()
cleaned_df.head()

,comment
0,"hallo warga sipil sekalian, selamat menonton"
1,prestasi menteri pendidikan lgbt ala barat
2,semoga mentri pendidikan periode ini bisa meng...
3,"jgn ngerokok bos, minimal di video ngga keliat..."
4,gen produk nadiem😅😅😅


In [124]:
# menangani slang dan bacaan panjang
cleaned_df['comment'] = cleaned_df['comment'].apply(replace_slang)
cleaned_df['comment'] = cleaned_df['comment'].apply(replace_word_elongation)

In [125]:
# menghapus link
def remove_link(text):
    text_c = re.sub(r'http\S+|www\S+|https\S+|<a\s+href="[^"]*">.*?</a>', '', str(text), flags=re.MULTILINE)
    return text_c

cleaned_df['comment'] = cleaned_df['comment'].apply(remove_link)
cleaned_df.head(15)

,comment
0,"halo warga sipil sekalian, selamat menonton"
1,prestasi menteri pendidikan lgbt ala barat
2,semoga mentri pendidikan periode ini bisa meng...
3,"jangan ngerokok bos, minimal di video tidak ke..."
4,gen produk nadiem😅😅😅
5,pemerintah salah tapi rakyatnya juga enggak ma...
6,salahnya bukan di sistemnya tapi penerapannya....
7,sumpah itu sama saya kayanya... malu banget
8,"koreksi bang,. soundmu kadang bikin kaget"
9,"teorinya 1000, prakteknya 0, pejabat di indone..."


In [126]:
# menghapus angka
def remove_number(text):
  text_c = ''.join([char for char in text if not char.isdigit()])
  return text_c

cleaned_df['comment'] = cleaned_df['comment'].apply(remove_number)
cleaned_df.head()

,comment
0,"halo warga sipil sekalian, selamat menonton"
1,prestasi menteri pendidikan lgbt ala barat
2,semoga mentri pendidikan periode ini bisa meng...
3,"jangan ngerokok bos, minimal di video tidak ke..."
4,gen produk nadiem😅😅😅


In [127]:
# menghapus tanda baca
def remove_punctuation(text):
  punctuation_set = set(string.punctuation)
  text_c = ''.join(char for char in text if char not in punctuation_set)
  return text_c

cleaned_df['comment'] = cleaned_df['comment'].apply(remove_punctuation)
cleaned_df.head()

,comment
0,halo warga sipil sekalian selamat menonton
1,prestasi menteri pendidikan lgbt ala barat
2,semoga mentri pendidikan periode ini bisa meng...
3,jangan ngerokok bos minimal di video tidak kel...
4,gen produk nadiem😅😅😅


In [128]:
# menghapus emoji
def remove_emoji(text):
    text_c = ''.join(char for char in text if char not in emoji.EMOJI_DATA)
    return text_c

cleaned_df['comment'] = cleaned_df['comment'].astype(str).apply(remove_emoji)
cleaned_df.head()

,comment
0,halo warga sipil sekalian selamat menonton
1,prestasi menteri pendidikan lgbt ala barat
2,semoga mentri pendidikan periode ini bisa meng...
3,jangan ngerokok bos minimal di video tidak kel...
4,gen produk nadiem


In [129]:
# menghapus white space
cleaned_df['comment'] = cleaned_df['comment'].astype(str).apply(lambda x: x.strip())
cleaned_df['comment'] = cleaned_df['comment'].apply(lambda x: ' '.join(x.split()))

cleaned_df.head()

,comment
0,halo warga sipil sekalian selamat menonton
1,prestasi menteri pendidikan lgbt ala barat
2,semoga mentri pendidikan periode ini bisa meng...
3,jangan ngerokok bos minimal di video tidak kel...
4,gen produk nadiem


In [130]:
# daftar stopwords bahasa indonesia
stopwords_indonesia = set(stopwords.words('indonesian'))

# function
def remove_stopwords(text):
    tokens = word_tokenize(str(text))
    kata_penting = [kata for kata in tokens if kata.lower() not in stopwords_indonesia]
    text_c = ' '.join(kata_penting)
    return text_c

cleaned_df['comment'] = cleaned_df['comment'].apply(remove_stopwords)

cleaned_df.head(15)

,comment
0,halo warga sipil selamat menonton
1,prestasi menteri pendidikan lgbt ala barat
2,semoga mentri pendidikan periode mengubah sist...
3,ngerokok bos minimal video perokok idolakan co...
4,gen produk nadiem
5,pemerintah salah rakyatnya pakai otaknya belaj...
6,salahnya sistemnya penerapannya buktinya sekol...
7,sumpah kayanya malu banget
8,koreksi bang soundmu kadang bikin kaget
9,teorinya prakteknya pejabat indonesia korupsi


,comment
0,hallo warga sipil selamat menonton
1,prestasi menteri pendidikan lgbt ala barat
2,semoga mentri pendidikan periode mengubah sist...
3,jangan ngerokok bos minimal video tidak keliha...
4,gen produk nadiem
5,pemerintah salah rakyatnya tidak pakai otaknya...
6,salahnya sistemnya penerapannya buktinya sekol...
7,a href sumpah sama saya kayaknya malu bangettt
8,koreksi bang soundmu kadang bikin kaget
9,teorinya prakteknya pejabat indonesia korupsi


In [131]:
# menangani teks tawa

def hapus_tawa(text):
    return re.sub(r'\b(wk)+\b|\b(xi)+\b', '', text, flags=re.IGNORECASE)

cleaned_df['comment'] = cleaned_df['comment'].apply(hapus_tawa)
cleaned_df.head(20)

,comment
0,halo warga sipil selamat menonton
1,prestasi menteri pendidikan lgbt ala barat
2,semoga mentri pendidikan periode mengubah sist...
3,ngerokok bos minimal video perokok idolakan co...
4,gen produk nadiem
5,pemerintah salah rakyatnya pakai otaknya belaj...
6,salahnya sistemnya penerapannya buktinya sekol...
7,sumpah kayanya malu banget
8,koreksi bang soundmu kadang bikin kaget
9,teorinya prakteknya pejabat indonesia korupsi


In [132]:
# cek data kosong setelah dibersihkan 
cleaned_df.isnull().sum()

comment    0
dtype: int64